### Openpcdet

- Multi gpus based : 
    - python -m torch.distributed.launch --nproc_per_node 4 train.py --cfg_file cfgs/kitti_models/PVRCNNPlusPlus.yaml --launcher pytorch --extra_tag test
    - nproc_per_node : GPU개수에 맞게 사용할 GPU(GPU개수보다 많은 숫자 불가)
    - cfg_file : 학습 시킬 config 경로
    - launcher : 생략 시 Multi GPU 사용이 안됨. 0,1,2,3 모두가 잡히긴 하나 분산작업이 되지 않음
+ Error
    - Key_Error : road_plane -> Config에서 사용하는 모델의 yaml수정(USE_ROAD_PLANE : True -> False)
    - DatasetLoader 이중으로 할 경우 Distributed Gpu pid 계속해서 증가해서 Cuda_Memory_Err발생
+ Logger
    - logger.info(log내용 기록)

### Custom_Dataset

In [12]:
### PV_RCNN - pvrcnn.yaml
CLASS_NAMES: ['Pedestrian', 'Stroller', 'Cart','Wheelchair']

DATA_CONFIG:
    _BASE_CONFIG_: /data/test_model/mm/OpenPCDet/tools/cfgs/dataset_configs/sosai_dataset.yaml

MODEL:
    NAME: PVRCNN

    VFE:
        NAME: MeanVFE

    BACKBONE_3D:
        NAME: VoxelBackBone8x

    MAP_TO_BEV:
        NAME: HeightCompression
        NUM_BEV_FEATURES: 256

    BACKBONE_2D:
        NAME: BaseBEVBackbone

        LAYER_NUMS: [5, 5]
        LAYER_STRIDES: [1, 2]
        NUM_FILTERS: [128, 256]
        UPSAMPLE_STRIDES: [1, 2]
        NUM_UPSAMPLE_FILTERS: [256, 256]

    DENSE_HEAD:
        NAME: AnchorHeadSingle
        CLASS_AGNOSTIC: False

        USE_DIRECTION_CLASSIFIER: True
        DIR_OFFSET: 0.78539
        DIR_LIMIT_OFFSET: 0.0
        NUM_DIR_BINS: 2

        ANCHOR_GENERATOR_CONFIG: [
            {
                'class_name': 'Pedestrian',
                'anchor_sizes': [[0.4, 0.6, 1.8]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [0],
                'align_center': False,
                'feature_map_stride': 8,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            },
            {
                'class_name': 'Stroller',
                'anchor_sizes': [[0.8, 0.5, 1.0]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [0],
                'align_center': False,
                'feature_map_stride': 8,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            },
            {
                'class_name': 'Cart',
                'anchor_sizes': [[0.5, 0.3, 0.7]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [0],
                'align_center': False,
                'feature_map_stride': 8,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            },
            {
                'class_name': 'Wheelchair',
                'anchor_sizes': [[1.1, 0.6, 0.9]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [0],
                'align_center': False,
                'feature_map_stride': 8,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            },
        ]

        TARGET_ASSIGNER_CONFIG:
            NAME: AxisAlignedTargetAssigner
            POS_FRACTION: -1.0
            SAMPLE_SIZE: 512
            NORM_BY_NUM_EXAMPLES: False
            MATCH_HEIGHT: False
            BOX_CODER: ResidualCoder

        LOSS_CONFIG:
            LOSS_WEIGHTS: {
                'cls_weight': 1.0,
                'loc_weight': 2.0,
                'dir_weight': 0.2,
                'code_weights': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
            }

    PFE:
        NAME: VoxelSetAbstraction
        POINT_SOURCE: raw_points
        NUM_KEYPOINTS: 2048
        NUM_OUTPUT_FEATURES: 128
        SAMPLE_METHOD: FPS

        FEATURES_SOURCE: ['bev', 'x_conv1', 'x_conv2', 'x_conv3', 'x_conv4', 'raw_points']
        SA_LAYER:
            raw_points:
                MLPS: [[16, 16], [16, 16]]
                POOL_RADIUS: [0.4, 0.8]
                NSAMPLE: [16, 16]
            x_conv1:
                DOWNSAMPLE_FACTOR: 1
                MLPS: [[16, 16], [16, 16]]
                POOL_RADIUS: [0.4, 0.8]
                NSAMPLE: [16, 16]
            x_conv2:
                DOWNSAMPLE_FACTOR: 2
                MLPS: [[32, 32], [32, 32]]
                POOL_RADIUS: [0.8, 1.2]
                NSAMPLE: [16, 32]
            x_conv3:
                DOWNSAMPLE_FACTOR: 4
                MLPS: [[64, 64], [64, 64]]
                POOL_RADIUS: [1.2, 2.4]
                NSAMPLE: [16, 32]
            x_conv4:
                DOWNSAMPLE_FACTOR: 8
                MLPS: [[64, 64], [64, 64]]
                POOL_RADIUS: [2.4, 4.8]
                NSAMPLE: [16, 32]

    POINT_HEAD:
        NAME: PointHeadSimple
        CLS_FC: [256, 256]
        CLASS_AGNOSTIC: True
        USE_POINT_FEATURES_BEFORE_FUSION: True
        TARGET_CONFIG:
            GT_EXTRA_WIDTH: [0.2, 0.2, 0.2]
        LOSS_CONFIG:
            LOSS_REG: smooth-l1
            LOSS_WEIGHTS: {
                'point_cls_weight': 1.0,
            }

    ROI_HEAD:
        NAME: PVRCNNHead
        CLASS_AGNOSTIC: True

        SHARED_FC: [256, 256]
        CLS_FC: [256, 256]
        REG_FC: [256, 256]
        DP_RATIO: 0.3

        NMS_CONFIG:
            TRAIN:
                NMS_TYPE: nms_gpu
                MULTI_CLASSES_NMS: False
                NMS_PRE_MAXSIZE: 9000
                NMS_POST_MAXSIZE: 512
                NMS_THRESH: 0.8
            TEST:
                NMS_TYPE: nms_gpu
                MULTI_CLASSES_NMS: False
                NMS_PRE_MAXSIZE: 1024
                NMS_POST_MAXSIZE: 100
                NMS_THRESH: 0.7

        ROI_GRID_POOL:
            GRID_SIZE: 6
            MLPS: [[64, 64], [64, 64]]
            POOL_RADIUS: [0.8, 1.6]
            NSAMPLE: [16, 16]
            POOL_METHOD: max_pool

        TARGET_CONFIG:
            BOX_CODER: ResidualCoder
            ROI_PER_IMAGE: 128
            FG_RATIO: 0.5

            SAMPLE_ROI_BY_EACH_CLASS: True
            CLS_SCORE_TYPE: roi_iou

            CLS_FG_THRESH: 0.75
            CLS_BG_THRESH: 0.25
            CLS_BG_THRESH_LO: 0.1
            HARD_BG_RATIO: 0.8

            REG_FG_THRESH: 0.55

        LOSS_CONFIG:
            CLS_LOSS: BinaryCrossEntropy
            REG_LOSS: smooth-l1
            CORNER_LOSS_REGULARIZATION: True
            LOSS_WEIGHTS: {
                'rcnn_cls_weight': 1.0,
                'rcnn_reg_weight': 1.0,
                'rcnn_corner_weight': 1.0,
                'code_weights': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
            }

    POST_PROCESSING:
        RECALL_THRESH_LIST: [0.3, 0.5, 0.7]
        SCORE_THRESH: 0.1
        OUTPUT_RAW_SCORE: False

        EVAL_METRIC: kitti

        NMS_CONFIG:
            MULTI_CLASSES_NMS: False
            NMS_TYPE: nms_gpu
            NMS_THRESH: 0.1
            NMS_PRE_MAXSIZE: 4096
            NMS_POST_MAXSIZE: 500


OPTIMIZATION:
    BATCH_SIZE_PER_GPU: 2
    NUM_EPOCHS: 80

    OPTIMIZER: adam_onecycle
    LR: 0.01
    WEIGHT_DECAY: 0.01
    MOMENTUM: 0.9

    MOMS: [0.95, 0.85]
    PCT_START: 0.4
    DIV_FACTOR: 10
    DECAY_STEP_LIST: [35, 45]
    LR_DECAY: 0.1
    LR_CLIP: 0.0000001

    LR_WARMUP: False
    WARMUP_EPOCH: 1

    GRAD_NORM_CLIP: 10


SyntaxError: invalid syntax (<ipython-input-12-84c5bfd27d01>, line 4)

In [13]:
### Custom_Dataset
import os
import numpy as np

from torch.utils.data import Dataset

from pcdet.datasets.processor.point_feature_encoder import PointFeatureEncoder
from pcdet.datasets.augmentor.data_augmentor import DataAugmentor
from pcdet.datasets.processor.data_processor import DataProcessor

from collections import defaultdict

from pcdet.datasets.kitti.kitti_object_eval_python.eval import *

import copy


class Onss_Dataset(Dataset):
    def __init__(self, split_dir, mode='train'):

        assert mode in ['train', 'valid', 'test']
        self.mode = mode

        if self.mode == 'train':
            # split_path = './split/train.txt'
            split_path = split_dir + '/train.txt'
        elif self.mode == 'valid':
            # split_path = './split/valid.txt'
            split_path = split_dir + '/valid.txt'
        elif self.mode == 'test':
            # split_path = './split/test.txt'
            split_path = split_dir + '/test.txt'

        self.samples = [line.rstrip() for line in open(split_path).readlines()]  # folder path, file number
        # number of sample data
        self.num_samples = len(self.samples)

        self.class_names = ['Pedestrian', 'Stroller', 'Cart', 'Wheelchair']

    #     def get_image(self, idx):
    #         import cv2
    #         img_dir_path = self.samples[idx].split(' ')[0]
    #         img_idx = self.samples[idx].split(' ')[-1]
    #         img_file = img_dir_path + '/cam/c_%s.png'%(img_idx)
    #         assert os.path.exists(img_file)
    #         return cv2.imread(img_file)  # (H, W, 3) BGR mode

    def get_lidar(self, idx):
        '''
        < Return >
            np_lidar_points : lidar points // numpy, (n,4), (x,y,z,intensity)
        '''
        lidar_dir_path = os.path.join(self.samples[idx].split(':')[0], 'lidar/lidar')
        lidar_idx = self.samples[idx].split(':')[1]
        lidar_file = lidar_dir_path + '/%s.bin' % lidar_idx
        assert os.path.exists(lidar_file)

        return np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 4)

    #     def get_calib(self, idx):
    #         calib_dir_path = self.samples[idx].split(' ')[0]
    #         calib_file = calib_dir_path + '/calb/calibration.txt'
    #         assert os.path.exists(calib_file)
    #         # TODO: calibration matrix
    #         calib = [line.rstrip() for line in open(calib_file).readlines()]   # folder path, file number
    #         return calib

    def get_annos(self, idx):
        '''
        get lidar annotations
        < Retrun >
            objects
        '''
        label_dir_path = os.path.join(self.samples[idx].split(':')[0], 'lidar/lidar_label')
        label_idx = self.samples[idx].split(':')[1]
        label_file = label_dir_path + '/llab_%s.txt' % label_idx

        assert os.path.exists(label_file)

        with open(label_file, 'r') as f:
            lines = f.readlines()
            if lines != []:
                list_cls = [float(line.split(' ')[1]) + 1 for line in lines]
                list_pos = [np.array([float(line.split(' ')[2]), float(line.split(' ')[3]), float(line.split(' ')[4])],
                                     dtype='float32') for line in lines]
                list_lwh = [np.array([float(line.split(' ')[5]), float(line.split(' ')[6]), float(line.split(' ')[7])],
                                     dtype='float32') for line in lines]
                list_ori = [float(line.split(' ')[8]) for line in lines]  # orientation(heading)

                annos = {}
                annos['cls'] = np.array(list_cls, dtype=np.float32)
                annos['name'] = [self.class_names[int(id - 1)] for id in list_cls]
                annos['pos'] = np.concatenate([pos.reshape(1, 3) for pos in list_pos], axis=0)
                annos['lwh'] = np.concatenate([lwh.reshape(1, 3) for lwh in list_lwh], axis=0)
                annos['ori'] = np.array(list_ori, dtype=np.float32)
                annos['location'] = np.concatenate([pos.reshape(1, 3) for pos in list_pos], axis=0)
                annos['dimensions'] = np.concatenate([lwh.reshape(1, 3) for lwh in list_lwh], axis=0)
                annos['rotation_y'] = np.array(list_ori, dtype=np.float32)
            else:
                annos = None
        return annos

    def __len__(self):
        return self.num_samples


#######################################################
# pointpillar Dataset
#######################################################
class OnssDataset(Onss_Dataset):
    def __init__(self, split_dir, mode='train', dataset_cfg=None):
        super().__init__(split_dir=split_dir, mode=mode)

        self.dataset_cfg = dataset_cfg

        # Feature encoder
        self.point_cloud_range = np.array(self.dataset_cfg.POINT_CLOUD_RANGE, dtype=np.float32)
        self.point_feature_encoder = PointFeatureEncoder(
            self.dataset_cfg.POINT_FEATURE_ENCODING,
            point_cloud_range=self.point_cloud_range
        )

        # Not use augmentor
        self.data_augmentor = None

        if self.mode == 'train':
            # create data processor
            self.data_processor = DataProcessor(
                self.dataset_cfg.DATA_PROCESSOR, point_cloud_range=self.point_cloud_range, training=True,
                num_point_features=3
            )
        else:
            self.data_processor = DataProcessor(
                self.dataset_cfg.DATA_PROCESSOR, point_cloud_range=self.point_cloud_range, training=False
            )

        self.grid_size = self.data_processor.grid_size
        self.voxel_size = self.data_processor.voxel_size

        self.total_epochs = 0

    def prepare_data(self, data_dict):
        """
        Args:
            data_dict:
                points: (N, 3 + C_in)
                gt_names: optional, (N), int
                gt_boxes: optional, (N, 7 + C) [x, y, z, dx, dy, dz, heading, ...]
                ...

        Returns:
            data_dict:
                frame_id: string
                points: (N, 3 + C_in)
                gt_names: optional, (N), int
                gt_boxes: optional, (N, 7 + C) [x, y, z, dx, dy, dz, heading, ...]

                use_lead_xyz: bool
                voxels: optional (num_voxels, max_points_per_voxel, 3 + C)
                voxel_coords: optional (num_voxels, 3)
                voxel_num_points: optional (num_voxels)
                ...
        """
        if self.mode != 'test':
            if len(data_dict['gt_boxes']) == 0:
                new_index = np.random.randint(self.__len__())
                return self.__getitem__(new_index)

        data_dict = self.point_feature_encoder.forward(data_dict)

        # Data Processing
        data_dict = self.data_processor.forward(data_dict=data_dict)  # return voxels, voxel_coords, voxel_num_points

        return data_dict

    def __getitem__(self, index):
        # Load data
        points = self.get_lidar(index)
        if self.mode == 'test':
            annos = None
        else:
            annos = self.get_annos(index)

        if annos is not None:
            gt_names = annos['cls']
            # gt_boxes: [pos_x, pos_y, pos_x, length, width, height, heading, class]
            tmp_gt_boxes = np.hstack((annos['pos'], annos['lwh']))
            gt_boxes = np.hstack((tmp_gt_boxes, annos['ori'].reshape(-1, 1)))
            gt_boxes = np.hstack((gt_boxes, gt_names.reshape(-1, 1))).astype(np.float32)

            location = annos['pos']
            dimensions = annos['lwh']
            rotation_y = annos['ori']

            # train and valid
            input_dict = {'frame_id': self.samples[index],
                          'points': points,
                          'gt_boxes': gt_boxes,
                          'location': location,
                          'dimensions': dimensions,
                          'rotation_y': rotation_y
                          }
        else:
            # test
            input_dict = {'frame_id': self.samples[index],
                          'points': points,
                          'gt_boxes': []}

        # prepare data
        data_dict = self.prepare_data(data_dict=input_dict)

        return data_dict

    @staticmethod
    def collate_batch(batch_list, _unused=False):
        data_dict = defaultdict(list)
        for cur_sample in batch_list:
            for key, val in cur_sample.items():
                data_dict[key].append(val)
        batch_size = len(batch_list)
        ret = {}

        for key, val in data_dict.items():
            try:
                if key in ['voxels', 'voxel_num_points']:
                    ret[key] = np.concatenate(val, axis=0)
                elif key in ['points', 'voxel_coords']:
                    coors = []
                    for i, coor in enumerate(val):
                        coor_pad = np.pad(coor, ((0, 0), (1, 0)), mode='constant', constant_values=i)
                        coors.append(coor_pad)
                    ret[key] = np.concatenate(coors, axis=0)
                elif key in ['gt_boxes']:
                    max_gt = max([len(x) for x in val])
                    batch_gt_boxes3d = np.zeros((batch_size, max_gt, val[0].shape[-1]), dtype=np.float32)
                    for k in range(batch_size):
                        batch_gt_boxes3d[k, :val[k].__len__(), :] = val[k]
                    ret[key] = batch_gt_boxes3d
                elif key in ['location', 'dimensions', 'rotation_y']:
                    ret[key] = np.concatenate(val, axis=0)

                else:
                    ret[key] = np.stack(val, axis=0)
            except:
                print('Error in collate_batch: key=%s' % key)
                raise TypeError

        ret['batch_size'] = batch_size
        return ret

    @staticmethod
    def generate_prediction_dicts(batch_dict, pred_dicts, class_names, output_path=None):
        """
        Args:
            batch_dict:
                frame_id:
            pred_dicts: list of pred_dicts
                pred_boxes: (N, 7), Tensor
                pred_scores: (N), Tensor
                pred_labels: (N), Tensor
            class_names:
            output_path:

        Returns:

        """

        def get_template_prediction(num_samples):
            ret_dict = {
                'name': np.zeros(num_samples), 'score': np.zeros(num_samples),
                'boxes_lidar': np.zeros([num_samples, 7])
            }
            return ret_dict

        def generate_single_sample_dict(box_dict):
            pred_scores = box_dict['pred_scores'].cpu().numpy()
            pred_boxes = box_dict['pred_boxes'].cpu().numpy()
            pred_labels = box_dict['pred_labels'].cpu().numpy()
            pred_dict = get_template_prediction(pred_scores.shape[0])
            if pred_scores.shape[0] == 0:
                return pred_dict

            pred_dict['name'] = np.array(class_names)[pred_labels - 1]
            pred_dict['score'] = pred_scores
            pred_dict['boxes_lidar'] = pred_boxes
            pred_dict['location'] = pred_boxes[:, 0:3]
            pred_dict['dimensions'] = pred_boxes[:, 3:6]
            pred_dict['rotation_y'] = pred_boxes[:, 6]

            return pred_dict

        annos = []
        for index, box_dict in enumerate(pred_dicts):
            single_pred_dict = generate_single_sample_dict(box_dict)
            single_pred_dict['frame_id'] = batch_dict['frame_id'][index]
            # single_pred_dict['metadata'] = batch_dict['metadata'][index]
            annos.append(single_pred_dict)

        return annos

    def clean_data(self, gt_anno, dt_anno, current_class, difficulty):
        CLASS_NAMES = ['Pedestrian', 'Stroller', 'Cart', 'Wheelchair']
        MIN_HEIGHT = [40, 25, 25]
        MAX_OCCLUSION = [0, 1, 2]
        MAX_TRUNCATION = [0.15, 0.3, 0.5]
        dc_bboxes, ignored_gt, ignored_dt = [], [], []
        current_cls_name = CLASS_NAMES[current_class].lower()
        num_gt = len(gt_anno["name"])
        num_dt = len(dt_anno["name"])
        num_valid_gt = 0

        for i in range(num_gt):
            gt_name = gt_anno["name"][i].lower()
            valid_class = -1
            if (gt_name == current_cls_name):
                valid_class = 1
            else:
                valid_class = -1

            ignore = False
            if valid_class == 1 and not ignore:
                ignored_gt.append(0)
                num_valid_gt += 1
            elif (valid_class == 0 or (ignore and (valid_class == 1))):
                ignored_gt.append(1)
            else:
                ignored_gt.append(-1)

        for i in range(num_dt):
            if (dt_anno["name"][i].lower() == current_cls_name):
                valid_class = 1
            else:
                valid_class = -1

            if valid_class == 1:
                ignored_dt.append(0)
            else:
                ignored_dt.append(-1)

        return num_valid_gt, ignored_gt, ignored_dt, dc_bboxes

    def _prepare_data(self, gt_annos, dt_annos, current_class, difficulty):
        gt_datas_list = []
        dt_datas_list = []
        total_dc_num = []
        ignored_gts, ignored_dets, dontcares = [], [], []
        total_num_valid_gt = 0
        for i in range(len(gt_annos)):
            rets = self.clean_data(gt_annos[i], dt_annos[i], current_class, difficulty)
            num_valid_gt, ignored_gt, ignored_det, dc_bboxes = rets

            ignored_gts.append(np.array(ignored_gt, dtype=np.int64))
            ignored_dets.append(np.array(ignored_det, dtype=np.int64))
            if len(dc_bboxes) == 0:
                dc_bboxes = np.zeros((0, 4)).astype(np.float64)
            else:
                dc_bboxes = np.stack(dc_bboxes, 0).astype(np.float64)
            total_dc_num.append(dc_bboxes.shape[0])
            dontcares.append(dc_bboxes)
            total_num_valid_gt += num_valid_gt
            gt_datas = np.concatenate([gt_annos[i]["pos"], gt_annos[i]["cls"][..., np.newaxis]], 1)
            dt_datas = np.concatenate([
                dt_annos[i]["score"][..., np.newaxis]
            ], 1)
            gt_datas_list.append(gt_datas)
            dt_datas_list.append(dt_datas)
        total_dc_num = np.stack(total_dc_num, axis=0)
        return (gt_datas_list, dt_datas_list, ignored_gts, ignored_dets, dontcares,
                total_dc_num, total_num_valid_gt)

    def eval_class(self,
                   gt_annos,
                   dt_annos,
                   current_classes,
                   difficultys,
                   metric,
                   min_overlaps,
                   compute_aos=False,
                   num_parts=100):
        """Kitti eval. support 2d/bev/3d/aos eval. support 0.5:0.05:0.95 coco AP.
        Args:
            gt_annos: dict, must from get_label_annos() in kitti_common.py
            dt_annos: dict, must from get_label_annos() in kitti_common.py
            current_classes: list of int, 0: car, 1: pedestrian, 2: cyclist
            difficultys: list of int. eval difficulty, 0: easy, 1: normal, 2: hard
            metric: eval type. 0: bbox, 1: bev, 2: 3d
            min_overlaps: float, min overlap. format: [num_overlap, metric, class].
            num_parts: int. a parameter for fast calculate algorithm

        Returns:
            dict of recall, precision and aos
        """
        assert len(gt_annos) == len(dt_annos)

        num_examples = len(gt_annos)
        split_parts = get_split_parts(num_examples, num_parts)

        rets = calculate_iou_partly(gt_annos, dt_annos, metric, num_parts)
        overlaps, parted_overlaps, total_dt_num, total_gt_num = rets
        N_SAMPLE_PTS = 41

        num_minoverlap = len(min_overlaps)
        num_class = len(current_classes)
        num_difficulty = len(difficultys)

        precision = np.zeros([num_class, num_difficulty, num_minoverlap, N_SAMPLE_PTS])
        recall = np.zeros([num_class, num_difficulty, num_minoverlap, N_SAMPLE_PTS])
        aos = np.zeros([num_class, num_difficulty, num_minoverlap, N_SAMPLE_PTS])

        for m, current_class in enumerate(current_classes):
            for l, difficulty in enumerate(difficultys):
                rets = self._prepare_data(gt_annos, dt_annos, current_class, difficulty)
                (gt_datas_list, dt_datas_list, ignored_gts, ignored_dets,
                 dontcares, total_dc_num, total_num_valid_gt) = rets
                for k, min_overlap in enumerate(min_overlaps[:, metric, m]):
                    thresholdss = []
                    for i in range(len(gt_annos)):
                        rets = compute_statistics_jit(
                            overlaps[i],
                            gt_datas_list[i],
                            dt_datas_list[i],
                            ignored_gts[i],
                            ignored_dets[i],
                            dontcares[i],
                            metric,
                            min_overlap=min_overlap,
                            thresh=0.0,
                            compute_fp=False)
                        _, _, _, _, thresholds = rets
                        thresholdss += thresholds.tolist()
                    thresholdss = np.array(thresholdss)
                    thresholds = get_thresholds(thresholdss, total_num_valid_gt)
                    thresholds = np.array(thresholds)

                    pr = np.zeros([len(thresholds), 4])
                    idx = 0
                    for j, num_part in enumerate(split_parts):
                        gt_datas_part = np.concatenate(gt_datas_list[idx:idx + num_part], 0)
                        dt_datas_part = np.concatenate(dt_datas_list[idx:idx + num_part], 0)
                        dc_datas_part = np.concatenate(dontcares[idx:idx + num_part], 0)
                        ignored_dets_part = np.concatenate(ignored_dets[idx:idx + num_part], 0)
                        ignored_gts_part = np.concatenate(ignored_gts[idx:idx + num_part], 0)
                        fused_compute_statistics(
                            parted_overlaps[j],
                            pr,
                            total_gt_num[idx:idx + num_part],
                            total_dt_num[idx:idx + num_part],
                            total_dc_num[idx:idx + num_part],
                            gt_datas_part,
                            dt_datas_part,
                            dc_datas_part,
                            ignored_gts_part,
                            ignored_dets_part,
                            metric,
                            min_overlap=min_overlap,
                            thresholds=thresholds,
                            compute_aos=compute_aos)
                        idx += num_part

                    for i in range(len(thresholds)):
                        recall[m, l, k, i] = pr[i, 0] / (pr[i, 0] + pr[i, 2])
                        precision[m, l, k, i] = pr[i, 0] / (pr[i, 0] + pr[i, 1])
                        if compute_aos:
                            aos[m, l, k, i] = pr[i, 3] / (pr[i, 0] + pr[i, 1])
                    for i in range(len(thresholds)):
                        precision[m, l, k, i] = np.max(precision[m, l, k, i:], axis=-1)
                        recall[m, l, k, i] = np.max(recall[m, l, k, i:], axis=-1)
                        if compute_aos:
                            aos[m, l, k, i] = np.max(aos[m, l, k, i:], axis=-1)
        ret_dict = {
            "recall": recall,
            "precision": precision,
            "orientation": aos,
        }
        return ret_dict

    def evaluation(self, det_annos, class_names, **kwargs):

        dt_annos = copy.deepcopy(det_annos)
        gt_annos = [copy.deepcopy(self.get_annos(idx)) for idx in range(self.num_samples)]

        current_classes = self.class_names
        overlap_0_5 = np.array([[0.3, 0.3, 0.3, 0.3],  # for easy [Pedestrian, Stroller, Cart, Wheelchair]
                                [0.3, 0.3, 0.3, 0.3],  # for moderate [Pedestrian, Stroller, Cart, Wheelchair]
                                [0.3, 0.3, 0.3, 0.3]])  # for hard [Pedestrian, Stroller, Cart, Wheelchair]
        min_overlaps = np.stack([overlap_0_5], axis=0)
        class_to_name = {
            0: 'Pedestrian',
            1: 'Stroller',
            2: 'Cart',
            3: 'Wheelchair'
        }
        name_to_class = {v: n for n, v in class_to_name.items()}
        if not isinstance(current_classes, (list, tuple)):
            current_classes = [current_classes]
        current_classes_int = []
        for curcls in current_classes:
            if isinstance(curcls, str):
                current_classes_int.append(name_to_class[curcls])
            else:
                current_classes_int.append(curcls)
        current_classes = current_classes_int
        min_overlaps = min_overlaps[:, :, current_classes]
        result = '\n'

        difficultys = [0, 1, 2]
        ret = self.eval_class(gt_annos, dt_annos, current_classes, difficultys, 2, min_overlaps)
        mAP3d = get_mAP(ret["precision"])

        ret_dict = {}
        for j, curcls in enumerate(current_classes):
            # mAP threshold array: [num_minoverlap, metric, class]
            # mAP result: [num_class, num_diff, num_minoverlap]
            for i in range(min_overlaps.shape[0]):
                result += print_str(
                    (f"{class_to_name[curcls]} "
                     "AP@{:.2f}:".format(min_overlaps[i, 2, j])))
                result += print_str((f"3d   AP:{mAP3d[j, 2, i]:.4f}, "))
        return result, ret_dict

ModuleNotFoundError: No module named 'pcdet'

In [ ]:
### Custom_train.py
import argparse
import datetime
import glob
import os
from pathlib import Path
from test import repeat_eval_ckpt

import torch
import torch.distributed as dist
import torch.nn as nn
from tensorboardX import SummaryWriter

from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models import build_network, model_fn_decorator
from pcdet.utils import common_utils
from train_utils.optimization import build_optimizer, build_scheduler
from train_utils.train_utils import train_model

from torch.utils.data import DataLoader
from pcdet.datasets.onss.onss_dataset import OnssDataset

### parser추가 가능(원하는 기능을 추가로 필요 시)
def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default=None, help='specify the config for training')

    parser.add_argument('--batch_size', type=int, default=None, required=False, help='batch size for training')
    parser.add_argument('--epochs', type=int, default=None, required=False, help='number of epochs to train for')
    parser.add_argument('--workers', type=int, default=8, help='number of workers for dataloader')
    parser.add_argument('--extra_tag', type=str, default='default', help='extra tag for this experiment')
    parser.add_argument('--ckpt', type=str, default=None, help='checkpoint to start from')
    parser.add_argument('--pretrained_model', type=str, default=None, help='pretrained_model')
    parser.add_argument('--launcher', choices=['none', 'pytorch', 'slurm'], default='none')
    parser.add_argument('--tcp_port', type=int, default=18888, help='tcp port for distrbuted training')
    parser.add_argument('--sync_bn', action='store_true', default=False, help='whether to use sync bn')
    parser.add_argument('--fix_random_seed', action='store_true', default=False, help='')
    parser.add_argument('--ckpt_save_interval', type=int, default=1, help='number of training epochs')
    parser.add_argument('--local_rank', type=int, default=0, help='local rank for distributed training')
    parser.add_argument('--max_ckpt_save_num', type=int, default=30, help='max number of saved checkpoint')
    parser.add_argument('--merge_all_iters_to_one_epoch', action='store_true', default=False, help='')
    parser.add_argument('--set', dest='set_cfgs', default=None, nargs=argparse.REMAINDER,
                        help='set extra config keys if needed')
    parser.add_argument('--max_waiting_mins', type=int, default=0, help='max waiting minutes')
    parser.add_argument('--start_epoch', type=int, default=0, help='')
    parser.add_argument('--save_to_file', action='store_true', default=False, help='')

    args = parser.parse_args()

    cfg_from_yaml_file(args.cfg_file, cfg)
    cfg.TAG = Path(args.cfg_file).stem
    cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'

    if args.set_cfgs is not None:
        cfg_from_list(args.set_cfgs, cfg)

    return args, cfg


def main():
    args, cfg = parse_config()
    if args.launcher == 'none':
        dist_train = False
        total_gpus = 1
    else:
        total_gpus, cfg.LOCAL_RANK = getattr(common_utils, 'init_dist_%s' % args.launcher)(
            args.tcp_port, args.local_rank, backend='nccl'
        )
        dist_train = True

    if args.batch_size is None:
        args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
    else:
        assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
        args.batch_size = args.batch_size // total_gpus

    args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs

    if args.fix_random_seed:
        common_utils.set_random_seed(666+cfg.LOCAL_RANK)

    output_dir = cfg.ROOT_DIR / 'output' / cfg.EXP_GROUP_PATH / cfg.TAG / args.extra_tag
    ckpt_dir = output_dir / 'ckpt'
    output_dir.mkdir(parents=True, exist_ok=True)
    ckpt_dir.mkdir(parents=True, exist_ok=True)

    log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

    # log to file
    logger.info('**********************Start logging**********************')
    gpu_list = os.environ['CUDA_VISIBLE_DEVICES'] if 'CUDA_VISIBLE_DEVICES' in os.environ.keys() else 'ALL'
    logger.info('CUDA_VISIBLE_DEVICES=%s' % gpu_list)

    if dist_train:
        logger.info('total_batch_size: %d' % (total_gpus * args.batch_size))
    for key, val in vars(args).items():
        logger.info('{:16} {}'.format(key, val))
    log_config_to_file(cfg, logger=logger)
    if cfg.LOCAL_RANK == 0:
        os.system('cp %s %s' % (args.cfg_file, output_dir))

    tb_log = SummaryWriter(log_dir=str(output_dir / 'tensorboard')) if cfg.LOCAL_RANK == 0 else None

    # train dataset load
    train_set = OnssDataset(split_dir='/extra_data/data/lidar_test/split/', mode='train', dataset_cfg=cfg.DATA_CONFIG)
    train_sampler = None
    train_loader = DataLoader(
        train_set, batch_size=args.batch_size, pin_memory=True, num_workers=args.workers,
        shuffle=True, collate_fn=train_set.collate_batch,
        drop_last=False, sampler=train_sampler, timeout=0
    )

    model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=train_set)
    if args.sync_bn:
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
    model.cuda()

    optimizer = build_optimizer(model, cfg.OPTIMIZATION)

    # load checkpoint if it is possible
    start_epoch = it = 0
    last_epoch = -1
    if args.pretrained_model is not None:
        model.load_params_from_file(filename=args.pretrained_model, to_cpu=dist, logger=logger)

    if args.ckpt is not None:
        it, start_epoch = model.load_params_with_optimizer(args.ckpt, to_cpu=dist, optimizer=optimizer, logger=logger)
        last_epoch = start_epoch + 1
    else:
        ckpt_list = glob.glob(str(ckpt_dir / '*checkpoint_epoch_*.pth'))
        if len(ckpt_list) > 0:
            ckpt_list.sort(key=os.path.getmtime)
            it, start_epoch = model.load_params_with_optimizer(
                ckpt_list[-1], to_cpu=dist, optimizer=optimizer, logger=logger
            )
            last_epoch = start_epoch + 1

    model.train()  # before wrap to DistributedDataParallel to support fixed some parameters
    if dist_train:
        model = nn.parallel.DistributedDataParallel(model, device_ids=[cfg.LOCAL_RANK % torch.cuda.device_count()])
    logger.info(model)

    lr_scheduler, lr_warmup_scheduler = build_scheduler(
        optimizer, total_iters_each_epoch=len(train_loader), total_epochs=args.epochs,
        last_epoch=last_epoch, optim_cfg=cfg.OPTIMIZATION
    )

    # -----------------------start training---------------------------
    logger.info('**********************Start training %s/%s(%s)**********************'
                % (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))

    train_model(
        model,
        optimizer,
        train_loader,
        model_func=model_fn_decorator(),
        lr_scheduler=lr_scheduler,
        optim_cfg=cfg.OPTIMIZATION,
        start_epoch=start_epoch,
        total_epochs=args.epochs,
        start_iter=it,
        rank=cfg.LOCAL_RANK,
        tb_log=tb_log,
        ckpt_save_dir=ckpt_dir,
        train_sampler=train_sampler,
        lr_warmup_scheduler=lr_warmup_scheduler,
        ckpt_save_interval=args.ckpt_save_interval,
        max_ckpt_save_num=args.max_ckpt_save_num,
        merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch
    )

    logger.info('**********************End training %s/%s(%s)**********************\n\n\n'
                % (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))

    logger.info('**********************Start evaluation %s/%s(%s)**********************' %
                (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))

    # test dataset load
    test_set = OnssDataset(split_dir='/extra_data/data/lidar_test/split/', mode='valid', dataset_cfg=cfg.DATA_CONFIG)
    test_sampler = None
    test_loader = DataLoader(
        test_set, batch_size=args.batch_size, pin_memory=True, num_workers=args.workers,
        shuffle=False, collate_fn=test_set.collate_batch,
        drop_last=False, sampler=test_sampler, timeout=0
    )

    eval_output_dir = output_dir / 'eval' / 'eval_with_train'
    eval_output_dir.mkdir(parents=True, exist_ok=True)
    args.start_epoch = max(args.epochs - 10, 0)  # Only evaluate the last 10 epochs

    repeat_eval_ckpt(
        model.module if dist_train else model,
        test_loader, args, eval_output_dir, logger, ckpt_dir,
        dist_test=dist_train
    )
    logger.info('**********************End evaluation %s/%s(%s)**********************' %
                (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))


if __name__ == '__main__':
    main()